---
draft: true
title: Finding pole positions
---

In [ ]:
# | code-fold: true
# | code-summary: Import Python libraries
from __future__ import annotations

from typing import Any

import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
from ampform.io import aslatex
from ampform.kinematics.phasespace import Kallen
from ampform.sympy import unevaluated
from iminuit import Minuit
from IPython.display import Math

## Theory

Define $T$ matrix via $K$-matrix:
$$T(s) = \left(\mathbb{I} + i K(s) {\rho_{cm}(s)}\right)^{-1} K(s)$$
For one channel and multiple resonances this gives:\
$$\sum_\mathrm{Res.}\frac{{K(s)}}{i {K(s)} {{\rho_{cm}(s)}} + 1}$$

A resonance pole in the complex $s$ plane occurs when the denominator of $T(s)$ goes to zero:
$$|T(s)| \rightarrow \infty \quad \Rightarrow \quad \left| \frac{1}{1 - i K(s)} \right|^2 \rightarrow \infty$$

Finding the poles is equivalent to minimizing:

$$\left|1 - i K(s)\right|^2 \rightarrow 0$$

to solve the problem numerically one can separate real and complex part of $s$ to construct a cost function which can be given to common minimizer: Minuit2
Cost function:
$$\text{Cost}(x, y) = \left|1 - i K(x + i y)\right|^2$$
using where $x$ and $y$ are minimization parameters.

## Symbolic implementation

In [ ]:
@unevaluated(real=False)
class PhaseSpaceFactor(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\rho\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return -16 * sp.pi * sp.I * ChewMandelstam(s, m1, m2)


@unevaluated(real=False)
class ChewMandelstam(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\Sigma\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        q = BreakupMomentum(s, m1, m2)
        return (
            1
            / (16 * sp.pi**2)
            * (
                (2 * q / sp.sqrt(s))
                * sp.log((m1**2 + m2**2 - s + 2 * sp.sqrt(s) * q) / (2 * m1 * m2))
                - (m1**2 - m2**2) * (1 / s - 1 / (m1 + m2) ** 2) * sp.log(m1 / m2)
            )
        )


@unevaluated(real=False)
class BreakupMomentum(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"q\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return sp.sqrt(Kallen(s, m1**2, m2**2)) / (2 * sp.sqrt(s))

In [ ]:
n = 1
K = sp.MatrixSymbol("K", n, n)
rho = sp.MatrixSymbol("rho", n, n)
I = sp.Identity(n)

In [ ]:
T1 = (I + sp.I * rho * K).inv() * K
T1

In [ ]:
T1.as_explicit()

In [ ]:
s, g1, m1, g2, m2 = sp.symbols("s g1 m1 g2 m2")
ma, mb = sp.symbols("ma mb")

In [ ]:
substitutions = {
    K[0, 0]: g1**2 / (m1**2 - s) + g2**2 / (m2**2 - s),
    rho[0, 0]: PhaseSpaceFactor(s, ma, mb),
}
Math(aslatex(substitutions))

In [ ]:
T1_expr = T1.as_explicit().subs(substitutions)[0].simplify(doit=False)
T1_expr

In [ ]:
rho = sp.MatrixSymbol("rho", n, n)
T2 = (T1.inv() + 2 * sp.I * rho).inv()

In [ ]:
T2_expr = T2.as_explicit().subs(substitutions)[0].simplify(doit=False)
T2_expr

In [ ]:
numerator, denominator = sp.fraction(T1_expr)
denominator

In [ ]:
parameters = {
    "m1": 1.8,
    "m2": 1.1,
    "g1": 0.5,
    "g2": 0.7,
    "ma": 0.1,
    "mb": 0.2,
}

In [ ]:
T1_func = sp.lambdify(s, T1_expr.doit().subs(parameters))
T2_func = sp.lambdify(s, T1_expr.doit().subs(parameters))

In [ ]:
Denom_func = sp.lambdify(s, denominator.doit().subs(parameters))

In [ ]:
x = np.linspace(0, 5, 500)
y = T2_func(x + 1e-8j)
plt.plot(x, np.abs(y))
plt.show()

In [ ]:
X, Y = np.meshgrid(
    np.linspace(-1, 6, num=100),
    np.linspace(1, -2, num=100),
)
S = X + 1j * Y
Z = T2_func(S)
mesh = plt.pcolormesh(X, Y, np.imag(Z), vmin=-1, vmax=1)
plt.colorbar(mesh)
plt.show()

In [ ]:
X, Y = np.meshgrid(
    np.linspace(0, 10, num=100),
    np.linspace(0, 10, num=100),
)
S = X + 1j * Y
Z = Denom_func(S)
mesh = plt.pcolormesh(X, Y, np.abs(Z))
plt.colorbar(mesh)
plt.show()

In [ ]:
sp.solve(sp.Eq(denominator.subs(parameters), 0), s)

## Numeric implementation

In [ ]:
resonances = [
    {"m0": 0.8, "g0": 0.4, "s_guess": complex(-1, 1)},
    {"m0": 1.0, "g0": 0.7, "s_guess": complex(-1, 1)},
]

In [ ]:
def rho_cm(s):
    m1 = 0.5
    m2 = 0.1
    s = complex(s)
    return np.sqrt((s - ((m1 + m2) ** 2)) * (s - (m1 - m2) ** 2) / s**2)

In [ ]:
def denominator(s, m0, g0):
    return s - m0**2 + 1j * g0**2 * rho_cm(s)


def pole_cost_function(s_re, s_im, m0, g0):
    s = complex(s_re, s_im)
    d = denominator(s, m0, g0)
    return d.real**2 + d.imag**2


def fit_pole(m0, g0, s_guess):
    def cost(s_re, s_im):
        return pole_cost_function(s_re, s_im, m0, g0)

    minuit2 = Minuit(cost, s_re=s_guess.real, s_im=s_guess.imag)
    minuit2.errordef = Minuit.LEAST_SQUARES
    return minuit2.migrad()

In [ ]:
m0_res1, g0_res1 = resonances[0]["m0"], resonances[0]["g0"]
s_guess_res1 = resonances[0]["s_guess"]
fit_res1 = fit_pole(m0_res1, g0_res1, s_guess_res1)
fit_res1

In [ ]:
m0_res2, g0_res2 = resonances[1]["m0"], resonances[1]["g0"]
s_guess_res2 = resonances[1]["s_guess"]
fit_res2 = fit_pole(m0_res2, g0_res2, s_guess_res2)
fit_res2